In [2]:
import scrapy

In [3]:
class Spider12(scrapy.Spider):
    name = 'Spider12' 
    allowed_domains = ['pagina12.com.ar']
    custom_settings = {'FEED_FORMAT': 'json', # Formato de archivo salida
                       'FEED_URI':'resultados.json', # archivo salida
                       'DEPTH_LIMIT': 2} # limite de repeticion
    
    start_urls = ['https://www.pagina12.com.ar/secciones/el-pais',
                  'https://www.pagina12.com.ar/secciones/economia',
                  'https://www.pagina12.com.ar/secciones/sociedad',
                  'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
                  'https://www.pagina12.com.ar/secciones/ciencia',
                  'https://www.pagina12.com.ar/secciones/el-mundo',
                  'https://www.pagina12.com.ar/secciones/deportes',
                  'https://www.pagina12.com.ar/secciones/contratapa']
    
    def parse(self, response):
        # Articulo promocionado
        nota_promocionada = response.xpath('//div[@class="featured-article__container"]/h2/a/@href').get()
        if nota_promocionada is not None:
            yield response.follow(nota_promocionada, callback=self.parse_nota)
        
        # Listado de notas
        notas = response.xpath('//ul[@class="article-list"]//li//a/@href').getall()
        for nota in notas:
            yield response.follow(nota, callback=self.parse_nota)
        
        # Link a la siguiente página   //*[@id="list-content"]/div[2]/div[2]/nav/a
        next_page = response.xpath('//*[@id="list-content"]/div[2]/div[2]/nav/a')
        #next_page = response.xpath('//a[@class="pagination-btn-next"]/@href')
        if next_page is not None:
            yield response.follow(next_page, callback=self.parse)
            
    def parse_nota(self, response):
        # Parseo la nota
        titulo = response.xpath('//div[@class="article-title"]/text()').get()
        cuerpo = ''.join(response.xpath('//div[@class="article-text"]/p/text()').getall())
        yield {'url': response.url,
               'titulo': titulo,
               'cuerpo': cuerpo}

In [4]:
from scrapy.crawler import CrawlerProcess



In [5]:
process = CrawlerProcess()
process.crawl(Spider12)
process.start()

 <GET https://www.pagina12.com.ar/secciones/el-pais> (referer: None)
2021-03-17 13:14:39 [scrapy.core.scraper] ERROR: Spider error processing <GET https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos/17-03-2021> (referer: None)
Traceback (most recent call last):
  File "/home/infinity/anaconda3/lib/python3.8/site-packages/scrapy/utils/defer.py", line 120, in iter_errback
    yield next(it)
  File "/home/infinity/anaconda3/lib/python3.8/site-packages/scrapy/utils/python.py", line 353, in __next__
    return next(self.data)
  File "/home/infinity/anaconda3/lib/python3.8/site-packages/scrapy/utils/python.py", line 353, in __next__
    return next(self.data)
  File "/home/infinity/anaconda3/lib/python3.8/site-packages/scrapy/core/spidermw.py", line 62, in _evaluate_iterable
    for r in iterable:
  File "/home/infinity/anaconda3/lib/python3.8/site-packages/scrapy/spidermiddlewares/offsite.py", line 29, in process_spider_output
    for x in result:
  File "/home/infinity/anaconda3